In [1]:
import wandb
wandb.login()
import tensorflow as tf
import pandas as pd
import mpra_model
from ray import air,tune
from ray.tune.integration.keras import TuneReportCallback
from ray.tune.search.hyperopt import HyperOptSearch
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.air.integrations.wandb import WandbLoggerCallback
import os
os.environ['CUDA_VISIBLE_DEVICES']='1'

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ambert. Use `wandb login --relogin` to force relogin
2023-05-04 17:58:48.467413: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-04 17:58:48.987858: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/ztang/.conda/envs/jax_tf/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
cell_type = 'HepG2'
RESULTS_DIR = '/home/ztang/multitask_RNA/evaluation/lenti_MPRA/results/'
data_dir = '/home/ztang/multitask_RNA/data/lenti_MPRA_embed/HepG2_2B5_1000G/'

config = {
    'input_shape': (41,2560),
    'reduce_dim':256,
    'conv1_filter': 256,
    'conv1_kernel':5,
    'activation':'exponential',
    'dropout1':0.2,
    'res_filter':3,
    'res_layers':2,
    'res_pool':4,
    'res_dropout':0.2,
    'conv2_filter':365,
    'conv2_kernel':3,
    'pool2_size':2,
    'dropout2':0.2,
    'dense':128,
    'dense2':64,
    'l_rate':0.0001
}


In [3]:
trainset = mpra_model.make_dataset(data_dir, 'train', mpra_model.load_stats(data_dir),
                            batch_size=128)
validset = mpra_model.make_dataset(data_dir, 'valid', mpra_model.load_stats(data_dir),
                            batch_size=128)

2023-05-04 17:58:53.301740: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 78865 MB memory:  -> device: 0, name: NVIDIA A100 80GB PCIe, pci bus id: 0000:47:00.0, compute capability: 8.0


In [4]:
def representation_training(config):
    
    model = mpra_model.rep_cnn(config['input_shape'],config)
    optimizer = tf.keras.optimizers.Adam(learning_rate=config['l_rate'])
    loss = tf.keras.losses.MeanSquaredError()
    model.compile(optimizer=optimizer,
                  loss=loss,
                  metrics=['mse'])
    model.summary()
    earlyStopping_callback = tf.keras.callbacks.EarlyStopping(
            patience=10, restore_best_weights=True
        )
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss', factor=0.2,
            patience=5, min_lr=1e-8)

    model.fit(
         trainset,
          epochs=100,
          batch_size=256,
          shuffle=False,
          validation_data=validset,
          callbacks=[earlyStopping_callback,reduce_lr
                     #,TuneReportCallback({"loss": "loss","val_loss":'val_loss'})
                     ]
      )

In [5]:
representation_training(config)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 sequence (InputLayer)          [(None, 41, 2560)]   0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 41, 256)      655616      ['sequence[0][0]']               
                                                                                                  
 conv1d_1 (Conv1D)              (None, 41, 256)      327936      ['conv1d[0][0]']                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 41, 256)     1024        ['conv1d_1[0][0]']               
 alization)                                                                                   

/home/ztang/.conda/envs/jax_tf/lib/python3.9/site-packages/keras/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(
2023-05-04 17:58:54.293511: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [219]
	 [[{{node Placeholder/_0}}]]
2023-05-04 17:58:54.293779: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for place

    875/Unknown - 30s 28ms/step - loss: 0.5381 - mse: 0.5381

2023-05-04 17:59:24.944784: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [28]
	 [[{{node Placeholder/_0}}]]
2023-05-04 17:59:24.944984: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [28]
	 [[{{node Placeholder/_0}}]]


875/875 [==============================] - 42s 42ms/step - loss: 0.5381 - mse: 0.5381 - val_loss: 0.6497 - val_mse: 0.6497 - lr: 1.0000e-04
Epoch 2/100
875/875 [==============================] - 36s 41ms/step - loss: 0.4709 - mse: 0.4709 - val_loss: 0.5036 - val_mse: 0.5036 - lr: 1.0000e-04
Epoch 3/100
875/875 [==============================] - 36s 41ms/step - loss: 0.4191 - mse: 0.4191 - val_loss: 0.5130 - val_mse: 0.5130 - lr: 1.0000e-04
Epoch 4/100
215/875 [======>.......................] - ETA: 18s - loss: 0.3939 - mse: 0.3939

KeyboardInterrupt: 

In [5]:
def tune_lentiMPRA(num_training_iterations):
    sched = AsyncHyperBandScheduler(
        time_attr="training_iteration", max_t=400, grace_period=20
    )

    tuner = tune.Tuner(
        tune.with_resources(representation_training, resources={"cpu": 4, "gpu": 1}),
        tune_config=tune.TuneConfig(
            search_alg = HyperOptSearch(),
            metric="val_loss",
            mode="min",
            scheduler=sched,
            num_samples=30,
        ),
        run_config=air.RunConfig(
            name="lentiMPRA_rep_training",
            stop={"training_iteration": num_training_iterations},
            callbacks=[WandbLoggerCallback(project="nuc_transfromer_lentiMPRA")]
        ),
        param_space=config,
    )
    results = tuner.fit()

    print("Best hyperparameters found were: ", results.get_best_result().config)

In [5]:
tune_lentiMPRA(num_training_iterations=1)

2023-04-27 19:40:03,222	INFO worker.py:1616 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
2023-04-27 19:40:11,580	INFO tune.py:218 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `Tuner(...)`.


2023-04-27 19:40:11,605	INFO wandb.py:315 -- Already logged into W&B.
(_WandbLoggingActor pid=497950) wandb: Currently logged in as: ambert. Use `wandb login --relogin` to force relogin
(pid=498008) 2023-04-27 19:40:14.509378: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
(_WandbLoggingActor pid=497950) wandb: Tracking run with wandb version 0.15.0
(_WandbLoggingActor pid=497950) wandb: Run data is saved locally in /om/home/ztang/ray_results/lentiMPRA_rep_training/representation_training_2e2d0aae_1_activation=exponential,conv1_filter=119,conv1_kernel=7,conv2_filter=108,conv2_kernel=6,dense=99_2023-04-27_19-40-11/wandb/run-20230427_194013-2e2d0aae
(_WandbLoggingActor pid=497950) wandb: Run `wandb offline` to turn off syncing.
(_WandbLoggingActor pid=497950) wandb: Syncing run representation_training_2e2d0aae
(_WandbLoggingActor pid=497950) wandb: ⭐️ View project at https://wandb.ai/ambert/nuc_transfromer_lentiMPRA
(_WandbLoggingActor pid

(representation_training pid=498008) Model: "model"
(representation_training pid=498008) _________________________________________________________________
(representation_training pid=498008)  Layer (type)                Output Shape              Param #   
(representation_training pid=498008) =================================================================
(representation_training pid=498008)  sequence (InputLayer)       [(None, 41, 2560)]        0         
(representation_training pid=498008)                                                                  
(representation_training pid=498008)  conv1d (Conv1D)             (None, 41, 119)           2132599   
(representation_training pid=498008)                                                                  
(representation_training pid=498008)  batch_normalization (BatchN  (None, 41, 119)          476       
(representation_training pid=498008)  ormalization)                                                   
(representation_train

(representation_training pid=498008) WARNING:tensorflow:Keras is training/fitting/evaluating on array-like data. Keras may not be optimized for this format, so if your input data format is supported by TensorFlow I/O (https://github.com/tensorflow/io) we recommend using that to load a Dataset instead.


  4/438 [..............................] - ETA: 14:41 - loss: 8.7355 - mse: 8.7355  


2023-04-27 19:40:39,874	WARNING tune.py:184 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 


In [4]:
smoke_test = True
algo = tune.search.bayesopt.BayesOptSearch()  # For testing purposes: set this to False to run the full experiment
analysis = tune.run(
    representation_training,
    name="lentiMPRA_rep_training",
    callbacks=[WandbLoggerCallback(project="nuc_transfromer_lentiMPRA")],
    scheduler=AsyncHyperBandScheduler(
        time_attr="training_iteration",
        max_t=400,
        grace_period=20
    ),
    search_alg= algo,
    metric="mse",
    mode="min",
    stop={
        "mse": 0.01,
        "training_iteration": 1 if smoke_test else 100
    },
    num_samples=2 if smoke_test else 50,
    resources_per_trial={
        "cpu": 4,
        "gpu": 1
    },
    config=config
)

AttributeError: module 'ray.tune.search' has no attribute 'bayesopt'

In [ ]:
RESULTS_DIR = '/home/ztang/multitask_RNA/evaluation/lenti_MPRA/results/'

pd.DataFrame.from_dict(
        analysis.best_config,
        orient='index'
).to_csv(
        RESULTS_DIR + 'tune_best.csv',
        header=True,
        index=True
        )
analysis.results_df.to_csv(RESULTS_DIR + 'tune_all.csv')